# Create Event Tables

The first thing we need to do is create tables that indicate whether a patient has the given event or not. A table should be created for each 

## Setup

In [1]:
import pandas as pd
import cnvutils
import os

In [2]:
def read_cancer_type_tcga(cancer_type):
    
    df = pd.read_csv(
        os.path.join("..", "data", cancer_type, "Gistic2_CopyNumber_Gistic2_all_thresholded.by_genes.gz"), 
        sep="\t"
    )
    
    return df

In [3]:
# List here the types of cancer we want to look at for this event

CANCER_TYPES = [
    'brca',
#     'CCRCC',
    'coad',
#     'ENDO',
#     'GBM',
    'hnsc',
    'luad',
    'lusc',
    'ov'
]

cnv_tables = {}
for cancer_type in CANCER_TYPES:
    cnv_tables[cancer_type] = read_cancer_type_tcga(cancer_type)

## Append Gene location Data

In [4]:
locations = cnvutils.get_gene_locations()

In [5]:
for cancer_type in cnv_tables.keys():
    
    df = cnv_tables[cancer_type].copy()
    idx_split = df["Gene Symbol"].str.split("|", n=1, expand=True)
    df = df.drop(columns="Gene Symbol")
    idx_split = idx_split.fillna("unknown")
    df = df.assign(
        Name=idx_split[0],
        Database_ID=idx_split[1]
    )
    df = df.set_index(["Name", "Database_ID"])
    
    df = df.join(locations)
    
    cnv_tables[cancer_type] = df

## Get percentage with event

In [6]:
CUTOFF = 0.8
# These are the event ranges we were using. I am leaving them here for now in case we find we need them again.
# GAIN_EVENT_START = 80794385
# GAIN_EVENT_END = 130794385
# LOSS_EVENT_START = 0
# LOSS_EVENT_END = 30794385

#These are the new event ranges (30 Sep 2020)
GAIN_EVENT_START = 52110839
GAIN_EVENT_END = 145052465
LOSS_EVENT_START = 202660
LOSS_EVENT_END = 37421341

In [7]:
def has_gain_event(row, cutoff):
    values = list(row)
    percent = len([x for x in values if x > 0]) / len(values)
    return percent >= cutoff

In [8]:
def has_loss_event(row, cutoff):
    values = list(row)
    percent = len([x for x in values if x < 0]) / len(values)
    return percent >= cutoff

In [9]:
for cancer_type in cnv_tables.keys():
    df = cnv_tables[cancer_type]
    # Subset Chromosome 8
    df_8 = df[df.chromosome == '8']
    # Find Gain Events
    event_gain = df_8[df_8.start_bp > GAIN_EVENT_START]
    event_gain = event_gain[event_gain.end_bp < GAIN_EVENT_END]
    event_gain = event_gain.drop(columns=['chromosome', 'start_bp', 'end_bp', 'arm'])
    gain_event = event_gain.apply(lambda x: has_gain_event(x, CUTOFF))
    # Find Loss Events
    event_loss = df_8[df_8.start_bp > LOSS_EVENT_START]
    event_loss = event_loss[event_loss.end_bp < LOSS_EVENT_END]
    event_loss = event_loss.drop(columns=['chromosome', 'start_bp', 'end_bp', 'arm'])
    loss_event = event_loss.apply(lambda x: has_loss_event(x, CUTOFF))
    # Create event table
    event_table = pd.DataFrame({'gain_event': gain_event, 'loss_event': loss_event})
    # Write to csv
    event_table.to_csv(f'{cancer_type}_has_event.tsv', sep='\t')

In [10]:
event_table.shape

(579, 2)

In [11]:
event_table["gain_event"].sum()

260

In [12]:
event_table["loss_event"].sum()

342

In [13]:
event_table

,gain_event,loss_event
TCGA-04-1331-01,False,False
TCGA-04-1332-01,True,True
TCGA-04-1335-01,True,False
TCGA-04-1336-01,False,False
TCGA-04-1337-01,False,False
TCGA-04-1338-01,True,True
TCGA-04-1341-01,False,False
TCGA-04-1342-01,True,False
TCGA-04-1343-01,True,False
TCGA-04-1346-01,False,True
